In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ferret import Benchmark
from statistics import mean
from tabulate import tabulate
import numpy as np

In [6]:
name = "/beegfs/scratch/rsari/BertSeqCA/checkpoint-10000/"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)

In [7]:
id2tag = {0: 'Anrede', 1: 'Diagnosen', 2: 'AllergienUnverträglichkeitenRisiken', 3: 'Anamnese', 4: 'Medikation', 5: 'KUBefunde', 6: 'Befunde', 7: 'EchoBefunde', 8: 'Zusammenfassung', 9: 'Mix', 10: 'Abschluss'}
tag2id = {tag: id for id, tag in id2tag.items()}

labels = list(id2tag.values())

In [8]:
import tensorflow as tf
import torch

2023-03-16 13:16:07.631261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 13:16:11.921508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /beegfs/homes/rsari/MIEdeep/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-16 13:16:11.921571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-16 13:16:12.350876: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has a

In [10]:
data, count = {l: [] for l in labels}, {l:0 for l in labels}

with open("../doctoral_letters/MIEdeep/data/PETsectionclass/50shot/set_1.tsv", "r") as f:
    lines = f.readlines()
    for line in lines:
        sent, lab = line.strip().split("\t")
        pred = torch.argmax(model(tokenizer.encode(sent, return_tensors="pt"))[0])
        l = id2tag[int(pred)]        
        if l==lab and count[lab]>=10:
            continue
        elif l!= lab:
            data[lab].append((l, sent))
        else:
            data[lab].append((l, sent))
            count[lab]+=1

In [12]:
import json

with open("ferret_ready_data.json", "w") as f:
    json.dump(data, f)

#### Score sentences with bench.explain for IG & SHAP

In [13]:
bench = Benchmark(model, tokenizer)

In [15]:
ig, shap = {l: [] for l in labels}, {l: [] for l in labels}

In [ ]:
from tqdm import tqdm

for lab in tqdm(labels):
    for tup in data[lab]:
        if tup[0] == lab:
            explanations = bench.explain(tup[1], target=tag2id[lab])                                    
            ig[lab].append(explanations[4])
            shap[lab].append(explanations[0])
        else:
            pass

In [58]:
assert all([i[0] == s[0] == d[0] for i,s,d in zip(ig.items(), shap.items(), data.items())])

In [59]:
eva_ig, eva_shap = {l: [] for l in labels}, {l: [] for l in labels}

In [60]:
for lab in tqdm(labels):
    for exp1, exp2 in zip(ig[lab], shap[lab]):
        eva_ig[lab].append(bench.evaluate_explanation(exp1, target = tag2id[lab]))
        eva_shap[lab].append(bench.evaluate_explanation(exp2, target = tag2id[lab]))

  0%|          | 0/11 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

...

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluator:   0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 11/11 [04:36<00:00, 25.09s/it]


In [61]:
[(k, len(v)) for k, v in eva_ig.items()]

[('Anrede', 20),
 ('Diagnosen', 10),
 ('AllergienUnverträglichkeitenRisiken', 10),
 ('Anamnese', 10),
 ('Medikation', 10),
 ('KUBefunde', 10),
 ('Befunde', 10),
 ('EchoBefunde', 10),
 ('Zusammenfassung', 10),
 ('Mix', 10),
 ('Abschluss', 10)]

In [62]:
import pickle

pickle.dump(data, open("data.p", "wb"))

pickle.dump(ig, open("ig.p", "wb"))
pickle.dump(shap, open("shap.p", "wb"))
            
pickle.dump(eva_ig, open("eva_ig.p", "wb"))
pickle.dump(eva_shap, open("eva_shap.p", "wb"))

In [86]:
eva_ig

{'Anrede': [ExplanationEvaluation(explanation=Explanation(text='über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 30/05/2007> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand.', tokens=['[CLS]', 'über', 'Ihren', 'Patienten', 'B', '-', 'SA', '##L', '##U', '##T', '##E', 'B', '-', 'P', '##ER', 'I', '-', 'P', '##ER', 'geboren', 'am', '<', '[', 'Pseud', '##o', ']', '30', '/', '05', '/', '2007', '>', 'wohn', '##haft', 'in', 'B', '-', 'P', '##L', '##Z', 'B', '-', 'L', '##OC', 'I', '-', 'A', '##D', '##DR', 'I', '-', 'A', '##D', '##DR', 'der', 'sich', 'vom', 'bis', 'in', 'unserer', 'stationären', 'Behandlung', 'befand', '.', '[SEP]'], scores=array([ 1.89367908e-02,  9.24629678e-02,  9.14723418e-02, -7.10023197e-02,
          4.82276671e-03, -1.06788352e-02, -1.13351063e-02,  4.44820733e-02,
          6.78783427e-03,  1.23417469e-02,  8.68712368e-03, -1.21738819e-02,
         -1.34678924e-02, -1.09962451e-02, -2.33171769e-02,  1.1485316

In [71]:
with open("data.p", "rb") as f:
    data = pickle.load(f)
    
[(k, len(v)) for k,v in data.items()]

[('Anrede', 11),
 ('Diagnosen', 10),
 ('AllergienUnverträglichkeitenRisiken', 13),
 ('Anamnese', 14),
 ('Medikation', 10),
 ('KUBefunde', 11),
 ('Befunde', 12),
 ('EchoBefunde', 17),
 ('Zusammenfassung', 13),
 ('Mix', 21),
 ('Abschluss', 16)]

In [72]:
with open("ig.p", "rb") as f:
    ig = pickle.load(f)
    
with open("shap.p", "rb") as f:
    shap = pickle.load(f)
    
with open("eva_ig.p", "rb") as f:
    eva_ig = pickle.load(f)
    
with open("eva_shap.p", "rb") as f:
    eva_shap = pickle.load(f)

In [74]:
table = [[l, mean([e.score for eva in eva_ig[l] for e in eva.evaluation_scores if e.name=="aopc_compr"]), mean([e.score for eva in eva_ig[l] for e in eva.evaluation_scores if e.name=="aopc_suff"])] for l in labels]
# Sort by Comprehensiveness
table1 = sorted(table, key= lambda x:x[1], reverse=True)
table1.insert(0, ["Label", "IG-Compr(sorted)", "IG-Suff"])

print(tabulate(table1, headers="firstrow"))
print("\n")
table2 = sorted(table, key= lambda x:x[2], reverse=True)
table2.insert(0, ["Label", "IG-Compr", "IG-Suff(sorted)"])

print(tabulate(table2, headers="firstrow"))

Label                                  IG-Compr(sorted)    IG-Suff
-----------------------------------  ------------------  ---------
Mix                                           0.643553    0.803807
Anrede                                        0.400537    0.713118
AllergienUnverträglichkeitenRisiken           0.313543    0.597452
Diagnosen                                     0.307299    0.661008
Abschluss                                     0.302274    0.450975
KUBefunde                                     0.301458    0.771444
EchoBefunde                                   0.254583    0.369241
Befunde                                       0.201851    0.405442
Zusammenfassung                               0.143771    0.436389
Medikation                                    0.130202    0.432135
Anamnese                                      0.0203175   0.462282


Label                                  IG-Compr    IG-Suff(sorted)
-----------------------------------  ----------  -----------

In [76]:
compr, suff = {l: None for l in ["Anrede"]}, {l: None for l in ["Anrede"]}

for l in labels:
    right = [data[l].index(d) for d in data[l] if d[0] == l]
    wrong = [data[l].index(d) for d in data[l] if d[0] != l]
    print(l, right, wrong)
    compr_ig = [e.score for eva in eva_ig[l] for e in eva.evaluation_scores if e.name=="aopc_compr"]
    compr_shap = [e.score for eva in eva_shap[l] for e in eva.evaluation_scores if e.name=="aopc_compr"]
    
    suff_ig = [e.score for eva in eva_ig[l] for e in eva.evaluation_scores if e.name=="aopc_suff"]
    suff_shap = [e.score for eva in eva_shap[l] for e in eva.evaluation_scores if e.name=="aopc_suff"]
    
    compr[l] = {"IG": (np.nanmean([e for e in compr_ig if compr_ig.index(e) in right]), np.nanmean([e for e in compr_ig if compr_ig.index(e) in wrong])), 
                "SHAP":(np.nanmean([e for e in compr_shap if compr_shap.index(e) in right]), np.nanmean([e for e in compr_shap if compr_shap.index(e) in wrong]))}
    suff[l] = {"IG": (np.nanmean([e for e in suff_ig if suff_ig.index(e) in right]), np.nanmean([e for e in suff_ig if suff_ig.index(e) in wrong])), 
                "SHAP":(np.nanmean([e for e in suff_shap if suff_shap.index(e) in right]), np.nanmean([e for e in suff_shap if suff_shap.index(e) in wrong]))}

Anrede [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [10]
Diagnosen [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] []
AllergienUnverträglichkeitenRisiken [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [10, 11, 12]
Anamnese [0, 1, 2, 3, 4, 6, 9, 10, 11, 12] [5, 7, 8, 13]
Medikation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] []
KUBefunde [0, 1, 2, 3, 5, 6, 7, 8, 9, 10] [4]
Befunde [0, 1, 2, 3, 4, 5, 6, 8, 9, 10] [7, 11]
EchoBefunde [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [10, 11, 12, 13, 14, 15, 16]
Zusammenfassung [0, 1, 2, 3, 4, 6, 7, 8, 9, 10] [5, 11, 12]
Mix [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] [0, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Abschluss [0, 2, 3, 4, 5, 6, 7, 8, 9, 11] [1, 10, 12, 13, 14, 15]


Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice


In [79]:
s = [(str(tup[0]), str(tup[1])) for k,v in compr.items() for tup in v.values()]
iter_s = iter(s)
string_compr = {l:{metr:next(iter_s) for metr in ["IG", "SHAP"]} for l in compr.keys()}

with open("ferret-compr-data.json", "w") as f:
    json.dump(string_compr, f)

s = [(str(tup[0]), str(tup[1])) for k,v in suff.items() for tup in v.values()]
iter_s = iter(s)
string_suff = {l:{metr:next(iter_s) for metr in ["IG", "SHAP"]} for l in suff.keys()}
              
with open("ferret-suff-data.json", "w") as f1:
    json.dump(string_suff, f1)

# Which of the interpretability methods is more faithful ?

**Approach:**
1. Calculate for IG & SHAP each Comprehensiveness & Sufficiency
    1. Which of them performs better in each ?
        1. Reasons?
    2. Which labels are conspicious for scoring high/low or 
        1. Metric specific tendencies ?
        2. Data Bias ?
    3. Finally IG or SHAP "better" overall ?
        1. Pros/Cons of approach with Faithfulness metric

## **1. Comprehensiveness**
$f(x)_j-f(x\text\r_j)_j$ → Score difference once most important tokens are removed.   
**Higher scores better: Exclusion of top 10-100% most important tokens does harm prediction.**
We expect performance on sentence to suddenly drop once 10% significant tokens removed.

### 1.1. SHAP

In [80]:
table = [(l, [[round(s, 2) for s in v] for k, v in compr[l].items() if k == "SHAP"]) for l in labels]
table = sorted(table, key = lambda x: x[1][0][0], reverse = True)
table.insert(0, ["Label", "Comprehensiveness mean scores"])

print(tabulate(table, headers="firstrow"))

Label                                Comprehensiveness mean scores
-----------------------------------  -------------------------------
Anrede                               [[1.0, nan]]
AllergienUnverträglichkeitenRisiken  [[0.85, nan]]
Mix                                  [[0.85, 0.99]]
Zusammenfassung                      [[0.82, 0.12]]
KUBefunde                            [[0.76, 0.93]]
Diagnosen                            [[0.75, nan]]
EchoBefunde                          [[0.66, nan]]
Anamnese                             [[0.65, 0.62]]
Befunde                              [[0.65, 0.79]]
Medikation                           [[0.64, nan]]
Abschluss                            [[0.58, 0.86]]


#### SHAP - Best Label: Anrede

In [81]:
len(eva_shap["Anrede"]), [(eva.explanation.text, eva.evaluation_scores[0]) for eva in eva_shap["Anrede"]]

(20,
 [('über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 30/05/2007> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand.',
   Evaluation(name='aopc_compr', score=0.99837464)),
  ('über Ihre Patientin B-SALUTE B-PER I-PER geboren am <[Pseudo] 18/06/1960> wh. B-PLZ B-LOC B-ADDR I-ADDR I-ADDR die sich vom bis in unserer stationären Behandlung befand.',
   Evaluation(name='aopc_compr', score=0.99882317)),
  ('über Ihre Patientin B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1978> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR die sich vom bis in unserer stationären Behandlung befand.',
   Evaluation(name='aopc_compr', score=0.99600077)),
  ('über Ihre Patientin B-SALUTE B-PER I-PER geboren am <[Pseudo] 15/09/1990> wh. B-PLZ B-LOC I-ADDR I-ADDR die sich am in unserer Ambulanz vorstellte.',
   Evaluation(name='aopc_compr', score=0.9971536)),
  ('über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 17/06/1986> wh. B-PLZ B-LOC B-AD

### 1.1.1. Manual Comprehensiveness Scores

**1. MASK TOKENS**

In [46]:
sent = data["Anrede"][9][1]
score = bench.score(sent)
target = tag2id["Anrede"]
metr = bench.explain(sent, target=target)[0] ### SHAP ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {tokens}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if i>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[PAD]"
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.argmax(list(bench.score(sentence).values()))}")
    aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Anrede
Original sentence: über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1977> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand. 	Score: 1.0
Filtered: ['über', 'Ihren', 'Patienten', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '##E', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'geboren', 'am', '<', '[', '[MASK]', '[MASK]', ']', '24', '/', '06', '/', '[MASK]', '[MASK]', 'wohn', '##haft', 'in', 'B', '-', 'P', '##L', '##Z', 'B', '-', 'L', '##OC', 'I', '-', '[MASK]', '[MASK]', '##DR', 'I', '-', 'A', '##D', '##DR', 'der', 'sich', '[MASK]', '[MASK]', 'in', 'unserer', 'stationären', 'Behandlung', 'befand', '.']

6 important token(s) removed: [PAD] [PAD] [PAD] B - SALUTE B - PER I - PER geboren am < [ Pseudo ] 24 / 06 / 1977 > [PAD]haft in B - PLZ B - LOC I - ADDR I - ADDR der sich vom bis in [PAD] stat

**2. DELETE TOKENS**

In [52]:
sent = data["Anrede"][9][1]
score = bench.score(sent)
target = tag2id["Anrede"]
metr = bench.explain(sent, target=target)[0] ### SHAP ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []
sor_scores = np.sort(scores)[::-1]

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0][:sect]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Anrede
Original sentence: über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1977> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand. 	Score: 1.0
Filtered: [('über', 0.27475452384179583), ('Ihren', 0.16060104808072384), ('Patienten', 0.09401567297402416), ('[MASK]', -0.009897939223456437), ('[MASK]', -0.005442124873400103), ('[MASK]', -0.03885965041934255), ('[MASK]', -0.02921427960125691), ('[MASK]', -0.014695494473159898), ('[MASK]', -0.008373574658791768), ('##E', 0.0018800988368467642), ('[MASK]', -0.008672825400404373), ('[MASK]', -0.0053829578243635955), ('[MASK]', -0.0038509972398680196), ('[MASK]', -0.006279497524352604), ('[MASK]', -0.004145131768475784), ('[MASK]', -0.004145131768475784), ('[MASK]', -0.001666349809875777), ('[MASK]', -0.003005235104904624), ('geboren', 0.029871898012828368), ('am', 0.00416953717002277), ('<', 0.003175417819

### Ablation Study:

In [111]:
sent = data["Anrede"][9][1]
score = bench.score(sent)
target = tag2id["Anrede"]
metr = bench.explain(sent, target=target)[0] ### SHAP ###
scores = [abs(i) for i in metr.scores[1:-1]]
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []
sor_scores = np.sort(scores)[::-1]

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0][:sect]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1977> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand. 	Score: 1.0
Filtered: [('über', 0.27475452384179583), ('Ihren', 0.16060104808072384), ('Patienten', 0.09401567297402416), ('B', 0.009897939223456437), ('-', 0.005442124873400103), ('SA', 0.03885965041934255), ('##L', 0.02921427960125691), ('##U', 0.014695494473159898), ('##T', 0.008373574658791768), ('##E', 0.0018800988368467642), ('B', 0.008672825400404373), ('-', 0.0053829578243635955), ('P', 0.0038509972398680196), ('##ER', 0.006279497524352604), ('I', 0.004145131768475784), ('-', 0.004145131768475784), ('P', 0.001666349809875777), ('##ER', 0.003005235104904624), ('geboren', 0.029871898012828368), ('am', 0.00416953717002277), ('<', 0.0031754178194350834), ('[', 0.0031754178194350834), ('Pseud', 0.003275748819466777), ('##o', 0.0007058410205896268), (']', 0.0004760277834258682), ('24', 0.00496373

<span style="color:purple">**! Inclusion of negative tokens doesn't improve score** </span>

In [54]:
sent = "über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1977> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand."
score = bench.score(sent)
target = tag2id["Anrede"]
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0][:sect]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: über Ihren Patienten B-SALUTE B-PER I-PER geboren am <[Pseudo] 24/06/1977> wohnhaft in B-PLZ B-LOC I-ADDR I-ADDR der sich vom bis in unserer stationären Behandlung befand. 	Score: 1.0
Filtered: [('über', 0.369749106852487), ('Ihren', 0.044333676676995296), ('[MASK]', -0.053847300736489696), ('[MASK]', -0.01390724980301558), ('[MASK]', -0.02504197542875501), ('[MASK]', -0.006230454071785075), ('##L', 0.03474725857720259), ('[MASK]', -0.001446737193103779), ('[MASK]', -0.01956262710407583), ('[MASK]', -0.02002716702890211), ('[MASK]', -0.008602941745248454), ('-', 0.0007182243532048894), ('P', 0.007569802764822435), ('[MASK]', -0.006974848328793949), ('I', 0.010918429309040334), ('[MASK]', -0.007757608511660943), ('P', 0.0037949314565197635), ('[MASK]', -0.0004361129416289774), ('[MASK]', -0.002481022254526685), ('[MASK]', -0.009243823314008581), ('[MASK]', -0.01804577132305862), ('[MASK]', -0.0048240117095126505), ('[MASK]', -1.7914025358376608e-06), ('[MASK]', -0.000

---

#### SHAP - Worst Label: Abschluss

In [38]:
len(eva_shap["Abschluss"]), [(eva.explanation.text, eva.evaluation_scores[0]) for eva in eva_shap["Abschluss"]]

(10,
 [('Zusätzliche Medikation nach haus- bzw. fachärztlicher Empfehlung.',
   Evaluation(name='aopc_compr', score=0.76513034)),
  ('I-PER / I.', Evaluation(name='aopc_compr', score=0.8592086)),
  ('Vorläufiger Brief!', Evaluation(name='aopc_compr', score=0.88514566)),
  ('Dieses Dokument wurde digital erstellt und ist ohne Unterschrift gültig.',
   Evaluation(name='aopc_compr', score=0.0122868335)),
  ('Wir entließen die kardiorespiratorisch stabile und beschwerdefreie Patientin auf ausdrücklichen Patientenwunsch bereits nach der farbduplexsonographischen Kontrolluntersuchung am <[Pseudo] 29/10/2036> in Ihre geschätzte ambulante Betreuung.',
   Evaluation(name='aopc_compr', score=0.58204925)),
  ('Wir danken für die Zusammenarbeit und verbleiben',
   Evaluation(name='aopc_compr', score=0.28277358)),
  ('Assistenzärzte', Evaluation(name='aopc_compr', score=0.34794673)),
  ('B-PER B-PER', Evaluation(name='aopc_compr', score=0.8227407)),
  ('Wir entließen den kardiorespiratorisch stabil

In [69]:
sent = data["Abschluss"][2][1]
target = tag2id["Abschluss"]
score = bench.score(sent)
metr = bench.explain(sent, target=target)[0] ### SHAP ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.max(list(bench.score(sentence).values()))]}: {np.argmax(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Abschluss
Original sentence: I-PER / I. 	Score: 1.0
Filtered: [('I', 0.25506598567669786), ('-', 0.028151328163901368), ('P', 0.4423505310834274), ('[MASK]', -0.1508690330826476), ('[MASK]', -0.009240054490267876), ('I', 0.07202195016845155), ('.', 0.04230095412692484)]

0 important token(s) removed: I - PER / I. 	 affects original sentence score: 0.0 | Labeled: Abschluss
1 important token(s) removed: I -ER / I. 	 affects original sentence score: 0.3 | Labeled: Abschluss
2 important token(s) removed: -ER / I. 	 affects original sentence score: 1.0 | Labeled: Befunde
2 important token(s) removed: -ER / I. 	 affects original sentence score: 1.0 | Labeled: Befunde
2 important token(s) removed: -ER / I. 	 affects original sentence score: 1.0 | Labeled: Befunde
3 important token(s) removed: -ER /. 	 affects original sentence score: 1.0 | Labeled: Befunde
4 important token(s) removed: -ER / 	 affects original sentence 

### Ablation Study:

In [120]:
sent = data["Abschluss"][2][1]
target = tag2id["Abschluss"]
score = bench.score(sent)
metr = bench.explain(sent, target=target)[0] ### SHAP ###
scores = [abs(i) for i in metr.scores[1:-1]]
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: I-PER / I. 	Score: 1.0
Filtered: [('I', 0.25506598567669786), ('-', 0.028151328163901368), ('P', 0.4423505310834274), ('##ER', 0.1508690330826476), ('/', 0.009240054490267876), ('I', 0.07202195016845155), ('.', 0.04230095412692484)]

1 important token(s) removed: I -ER / I. 	 affects original sentence score: 0.3 | Labeled: Abschluss: 0.6961265206336975
1 important token(s) removed: I -ER / I. 	 affects original sentence score: 0.3 | Labeled: Abschluss: 0.6961265206336975
2 important token(s) removed: -ER / I. 	 affects original sentence score: 1.0 | Labeled: Befunde: 0.9329257607460022
3 important token(s) removed: - / I. 	 affects original sentence score: 0.07 | Labeled: Abschluss: 0.9334296584129333
4 important token(s) removed: - /. 	 affects original sentence score: 0.17 | Labeled: Abschluss: 0.8323154449462891
4 important token(s) removed: - /. 	 affects original sentence score: 0.17 | Labeled: Abschluss: 0.8323154449462891
5 important token(s) removed: - / 	 af

In [119]:
bench.score("")

{'LABEL_0': 0.00016349625366274267,
 'LABEL_1': 0.00289585767313838,
 'LABEL_2': 0.001133674057200551,
 'LABEL_3': 0.20511578023433685,
 'LABEL_4': 0.009475763887166977,
 'LABEL_5': 0.0007412787526845932,
 'LABEL_6': 0.08826104551553726,
 'LABEL_7': 0.011516441591084003,
 'LABEL_8': 0.28758105635643005,
 'LABEL_9': 0.0030929988715797663,
 'LABEL_10': 0.3900226950645447}

<span style="color:purple">**! Inclusion of negative tokens had disadvantage: doesn't stray that much away from right prediction, such that correct label in upper half is sometimes predicted** </span>

In [70]:
sent = data["Abschluss"][2][1]
target = tag2id["Abschluss"]
score = bench.score(sent)
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.max(list(bench.score(sentence).values()))]}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Abschluss
Original sentence: I-PER / I. 	Score: 1.0
Filtered: [('[MASK]', -0.051868785885542876), ('[MASK]', -0.03826001713589519), ('P', 0.037529584085046766), ('##ER', 0.5228841551420212), ('[MASK]', -0.020026388897304626), ('I', 0.08666807399219652), ('[MASK]', -0.1685823047547219)]

0 important token(s) removed: I - PER / I. 	 affects original sentence score: 0.0 | Labeled: Abschluss
1 important token(s) removed: I - P / I. 	 affects original sentence score: 0.01 | Labeled: Abschluss
1 important token(s) removed: I - P / I. 	 affects original sentence score: 0.01 | Labeled: Abschluss
1 important token(s) removed: I - P / I. 	 affects original sentence score: 0.01 | Labeled: Abschluss
2 important token(s) removed: I - P /. 	 affects original sentence score: 0.04 | Labeled: Abschluss
2 important token(s) removed: I - P /. 	 affects original sentence score: 0.04 | Labeled: Abschluss
2 important token(s) removed:

### Ablation study

In [109]:
sent = data["Abschluss"][2][1]
target = tag2id["Abschluss"]
score = bench.score(sent)
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = [abs(i) for i in metr.scores[1:-1]]
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: I-PER / I. 	Score: 1.0
Filtered: [('I', 0.051868785885542876), ('-', 0.03826001713589519), ('P', 0.037529584085046766), ('##ER', 0.5228841551420212), ('/', 0.020026388897304626), ('I', 0.08666807399219652), ('.', 0.1685823047547219)]

1 important token(s) removed: I - P / I. 	 affects original sentence score: 0.01 | Labeled: Abschluss: 0.9940428137779236
1 important token(s) removed: I - P / I. 	 affects original sentence score: 0.01 | Labeled: Abschluss: 0.9940428137779236
2 important token(s) removed: I - P / I 	 affects original sentence score: 0.0 | Labeled: Abschluss: 0.9963114857673645
3 important token(s) removed: I - P / 	 affects original sentence score: 0.0 | Labeled: Abschluss: 0.997244119644165
4 important token(s) removed: - P / 	 affects original sentence score: 0.14 | Labeled: Abschluss: 0.8630074858665466
4 important token(s) removed: - P / 	 affects original sentence score: 0.14 | Labeled: Abschluss: 0.8630074858665466
5 important token(s) removed: P

**improvement**

### 1.2. IG

In [322]:
table = [(l, [[round(s, 2) for s in v] for k, v in compr[l].items() if k == "IG"]) for l in labels]
table = sorted(table, key = lambda x: x[1][0][0], reverse = True)
table.insert(0, ["Label", "Comprehensiveness mean scores"])

print(tabulate(table, headers="firstrow"))

Label                                Comprehensiveness mean scores
-----------------------------------  -------------------------------
Mix                                  [[0.64, -0.06]]
Anrede                               [[0.4, 0.01]]
Diagnosen                            [[0.31, nan]]
AllergienUnverträglichkeitenRisiken  [[0.31, 0.0]]
KUBefunde                            [[0.3, -0.0]]
Abschluss                            [[0.3, -0.06]]
EchoBefunde                          [[0.25, -0.02]]
Befunde                              [[0.2, -0.3]]
Zusammenfassung                      [[0.14, -0.09]]
Medikation                           [[0.13, nan]]
Anamnese                             [[0.02, -0.23]]


In [82]:
len(eva_ig["Mix"]), [(len(eva.explanation.tokens[1:-1]), eva.explanation.text, eva.evaluation_scores[0]) for eva in eva_ig["Mix"]]

(10,
 [(30,
   '- Wiedervorstellung zur nächsten Schrittmacherkontrolluntersuchung in unserer ORG am <[Pseudo] 05/11/2037> um 10 Uhr',
   Evaluation(name='aopc_compr', score=0.17364438)),
  (9,
   'Externer Zuweiser: CPU',
   Evaluation(name='aopc_compr', score=0.23609951)),
  (34,
   'Wir empfehlen regelmäßige angiologische Kontrolluntersuchungen in der Praxis des betreuenden B-SALUTE B-TITLE B-PER',
   Evaluation(name='aopc_compr', score=0.48343337)),
  (8,
   '- Kostaufbau nach Ernährungskonsil',
   Evaluation(name='aopc_compr', score=0.91242677)),
  (7,
   '- Übernahme Herzchirurgie',
   Evaluation(name='aopc_compr', score=0.6101352)),
  (12,
   '- Kaliumkontrolle und klinische Kontrolle morgen beim Hausarzt',
   Evaluation(name='aopc_compr', score=0.83366996)),
  (22,
   '- Termin in der kardiologischen Ambulanz am B-DATE, 10:30 Uhr',
   Evaluation(name='aopc_compr', score=0.97942823)),
  (11,
   '- AB weiter, regelmäßige Entzündungswertkontrollen',
   Evaluation(name='aopc_compr'

#### IG - Best Label: Mix

In [101]:
sent = data["Mix"][4][1]
score = bench.score(sent)
target = tag2id["Mix"]
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {tokens}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Befunde
Original sentence: - Kostaufbau nach Ernährungskonsil 	Score: 0.94
Filtered: ['[MASK]', 'Kost', '##aufbau', '[MASK]', 'Ernährung', '##skon', '##s', '##il']

1 important token(s) removed: -aufbau nach Ernährungskonsil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.984224796295166
1 important token(s) removed: -aufbau nach Ernährungskonsil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.984224796295166
2 important token(s) removed: -aufbau nach Ernährungskonil 	 affects original sentence score: 0.93 | Labeled: Befunde: 0.9672594666481018
2 important token(s) removed: -aufbau nach Ernährungskonil 	 affects original sentence score: 0.93 | Labeled: Befunde: 0.9672594666481018
3 important token(s) removed: -aufbau nach Ernährungil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.9623121619224548
4 important token(s) removed: - nach Ernährungil 	 affects original sentenc

### Ablation

In [121]:
sent = data["Mix"][4][1]
score = bench.score(sent)
target = tag2id["Mix"]
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = [abs(i) for i in metr.scores[1:-1]]
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {tokens}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(set(aggr)),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: - Kostaufbau nach Ernährungskonsil 	Score: 0.94
Filtered: ['-', 'Kost', '##aufbau', 'nach', 'Ernährung', '##skon', '##s', '##il']

1 important token(s) removed: -aufbau nach Ernährungskonsil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.984224796295166
2 important token(s) removed: -aufbau nach Ernährungskonil 	 affects original sentence score: 0.93 | Labeled: Befunde: 0.9672594666481018
2 important token(s) removed: -aufbau nach Ernährungskonil 	 affects original sentence score: 0.93 | Labeled: Befunde: 0.9672594666481018
3 important token(s) removed: ##aufbau nach Ernährungskonil 	 affects original sentence score: 0.93 | Labeled: Befunde: 0.9658105969429016
4 important token(s) removed: ##aufbau nach Ernährungil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.959843635559082
5 important token(s) removed: ##aufbau Ernährungil 	 affects original sentence score: 0.94 | Labeled: Befunde: 0.9578065872192383
6 important token(s) removed: Ern

#### IG - Worst Label: Anamnese

In [328]:
len(eva_ig["Anamnese"]), [(len(eva.explanation.tokens[1:-1]), eva.explanation.text, eva.evaluation_scores[0]) for eva in eva_ig["Anamnese"]]

(14,
 [(15,
   'Bei der Ankunft verneint der Patient Dyspnoe und AP-Beschwerden.',
   Evaluation(name='aopc_compr', score=0.001142174)),
  (10,
   'Er sei gerne zu Formel1-Rennen gegangen.',
   Evaluation(name='aopc_compr', score=0.00058760645)),
  (22,
   'Bislang habe sie sich gegen eine weitere Abklärung gesträubt, da sie im Alltag keine Beschwerden gehabt habe.',
   Evaluation(name='aopc_compr', score=0.0029604794)),
  (34,
   'Inadäquate Belastungsdyspnoe wird nicht beschrieben, jedoch bestünden seit ca. 2 Tagen belastungsinduzierte pectanginöse Beschwerden.',
   Evaluation(name='aopc_compr', score=0.00013010502)),
  (20,
   'Die ambulante Vorstellung erfolgt zur kardiologischen Abklärung bei bekannter arterieller Hypertonie.',
   Evaluation(name='aopc_compr', score=-0.0137945535)),
  (33,
   'Diffuse, von epikardial ausgehende, teilweise transmurale KM-Anreicherung im gesamten LV mit Betonung septal nachweisbar',
   Evaluation(name='aopc_compr', score=-0.00070842105)),
  (12,
   

In [127]:
sent = data["Anamnese"][1][1]
score = bench.score(sent)
target = tag2id[f"{data['Anamnese'][1][0]}"]
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = list(metr.scores[1:-1])
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(aggr),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

! Prediction swayed by removal of negative contributing tokens to label:  Anamnese
Original sentence: Er sei gerne zu Formel1-Rennen gegangen. 	Score: 1.0
Filtered: [('[MASK]', -0.10933393885356124), ('[MASK]', -0.34029973576430217), ('gerne', 0.17127102563532673), ('zu', 0.08689371458475474), ('Formel', 0.06380813653177972), ('[MASK]', -0.03776140815005633), ('[MASK]', -0.05917119274571068), ('[MASK]', -0.013922973211541031), ('gegangen', 0.0400934260616394), ('.', 0.04118801767227825)]

0 important token(s) removed: Er sei gerne zu Formel1 - Rennen gegangen. 	 affects original sentence score: 0.0 | Labeled: Anamnese: 0.9973194003105164
1 important token(s) removed: Er sei zu Formel1 - Rennen gegangen. 	 affects original sentence score: 0.0 | Labeled: Anamnese: 0.9967516660690308
2 important token(s) removed: Er sei Formel1 - Rennen gegangen. 	 affects original sentence score: 0.0 | Labeled: Anamnese: 0.9968478083610535
2 important token(s) removed: Er sei Formel1 - Rennen gegangen. 	

In [128]:
sent = data["Anamnese"][1][1]
score = bench.score(sent)
target = tag2id[f"{data['Anamnese'][1][0]}"]
metr = bench.explain(sent, target=target)[4] ### IG ###
scores = [abs(i) for i in metr.scores[1:-1]]
tokens = [t if scores[i]>=0 else "[MASK]" for i, t in enumerate(metr.tokens[1:-1])] 
try:
    assert score[f"LABEL_{target}"] <= bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens))))[f"LABEL_{target}"]
except:
    print("! Prediction swayed by removal of negative contributing tokens to label: ", 
          id2tag[np.argmax(list(bench.score(tokenizer.convert_tokens_to_string(list(filter(lambda x:x!="[MASK]", tokens)))).values()))])
aggr = []

print(f"Original sentence: {sent} \tScore: {round(score[f'LABEL_{target}'],2)}\nFiltered: {[(t, s) for t, s in zip(tokens, scores)]}\n")

for i in np.arange(.1, 1.1, .1):
    sect = round(len([s for s in scores if s>=0])*i)
    sor_scores = np.sort(scores)[::-1][:sect]
    sentence = metr.tokens[1:-1]
    real_i = [scores.index(s) for s in sor_scores if s>=0]
    for x in real_i:
        sentence[x] = "[MASK]"
    sentence = list(filter(lambda x: x!= "[MASK]", sentence))
    sentence = tokenizer.convert_tokens_to_string(sentence)
    new = score[f"LABEL_{target}"] - bench.score(sentence)[f"LABEL_{target}"]
    print(f"{sect} important token(s) removed: {sentence} \t affects original sentence score: {round(new, 2)} | Labeled: {id2tag[np.argmax(list(bench.score(sentence).values()))]}: {np.max(list(bench.score(sentence).values()))}")
    if sect >= 1:
        aggr.append(new)

print(f"\nMean of all scores: {round(mean(aggr),2)}")

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

Original sentence: Er sei gerne zu Formel1-Rennen gegangen. 	Score: 1.0
Filtered: [('Er', 0.10933393885356124), ('sei', 0.34029973576430217), ('gerne', 0.17127102563532673), ('zu', 0.08689371458475474), ('Formel', 0.06380813653177972), ('##1', 0.03776140815005633), ('-', 0.05917119274571068), ('Rennen', 0.013922973211541031), ('gegangen', 0.0400934260616394), ('.', 0.04118801767227825)]

1 important token(s) removed: Er gerne zu Formel1 - Rennen gegangen. 	 affects original sentence score: 0.0 | Labeled: Anamnese: 0.9931164979934692
2 important token(s) removed: Er zu Formel1 - Rennen gegangen. 	 affects original sentence score: 0.01 | Labeled: Anamnese: 0.991295576095581
3 important token(s) removed: zu Formel1 - Rennen gegangen. 	 affects original sentence score: 0.01 | Labeled: Anamnese: 0.987916886806488
4 important token(s) removed: Formel1 - Rennen gegangen. 	 affects original sentence score: 0.45 | Labeled: Anamnese: 0.546699583530426
5 important token(s) removed: ##1 - Rennen g

<span style="color:purple">**! Inclusion of negative tokens improves score such that Anamnese loses probability in upper half and flase positives are predicted in lower half** </span>